In [16]:
from torchxrayvision.datasets import NIH_Dataset
import torch
import numpy as np

In [24]:
class NIH_wrapper(NIH_Dataset):
    def __getitem__(self,idx):
        res = super().__getitem__(idx)
        res['label'] = res['lab']
        res['meta'] = {"age":np.array(self.csv["Patient Age"].iloc[idx]),
                        "gender":np.array((self.csv["Patient Gender"] == "M").iloc[idx])}
        return res

In [25]:
from torchsampler import ImbalancedDatasetSampler
ds = NIH_wrapper("/home/luab/experiments/data/nih_raw/",\
    csvpath="/home/luab/experiments/data/nih_raw/Data_Entry_2017_v2020.csv",\
    bbox_list_path="/home/luab/experiments/data/nih_raw/BBox_List_2017.csv",\
    unique_patients=True)
sampler=ImbalancedDatasetSampler(ds,callback_get_label=lambda x: x.labels[:,2])
dl = torch.utils.data.DataLoader(ds,batch_size=1000,sampler=sampler)

In [26]:
ds[0]

{'idx': 0,
 'lab': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32),
 'img': array([[[  598.3373 ,   574.24316,   542.1177 , ...,  -983.84314,
          -1007.93726, -1024.     ],
         [  574.24316,   550.14905,   542.1177 , ...,  -983.84314,
          -1007.93726, -1024.     ],
         [  550.14905,   534.0863 ,   526.05493, ...,  -983.84314,
          -1007.93726, -1024.     ],
         ...,
         [ 1024.     ,  1024.     ,  1024.     , ..., -1024.     ,
          -1024.     , -1024.     ],
         [ 1024.     ,  1024.     ,  1015.9686 , ..., -1024.     ,
          -1024.     , -1024.     ],
         [ 1024.     ,  1024.     ,  1024.     , ..., -1024.     ,
          -1024.     , -1024.     ]]], dtype=float32),
 'label': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32),
 'meta': {'age': array(57), 'gender': array(True)}}

In [32]:
i = 0
for j in dl:
    i +=1 

In [8]:
j.keys()

dict_keys(['idx', 'lab', 'img'])

In [29]:
from skimage.io import imread

In [30]:
imread("/home/luab/experiments/data/nih_raw/00015998_000.png")

ValueError: Could not find a backend to open `/home/luab/experiments/data/nih_raw/00015998_000.png`` with iomode `ri`.

In [7]:
!pip install torchsampler

In [2]:
import pandas as pd

In [4]:
nih = pd.read_csv("/data/nih_raw/Data_Entry_2017_v2020.csv")
nih_bbox = pd.read_csv("/data/nih_raw/BBox_List_2017.csv")
chexpert = pd.read_csv("/data/chexpert/CheXpert-v1.0-small/train.csv")

In [12]:
nih_bbox[nih_bbox['Finding Label'] == "Lung Opacity"]

,Image Index,Finding Label,Bbox [x,y,w,h],Unnamed: 6,Unnamed: 7,Unnamed: 8


In [22]:
nih['Finding Labels'].str.split("|")

0                    [Cardiomegaly]
1         [Cardiomegaly, Emphysema]
2          [Cardiomegaly, Effusion]
3                      [No Finding]
4                          [Hernia]
                    ...            
112115            [Mass, Pneumonia]
112116                 [No Finding]
112117                 [No Finding]
112118                 [No Finding]
112119                 [No Finding]
Name: Finding Labels, Length: 112120, dtype: object

In [20]:
chexpert[chexpert["Pleural Effusion"] == 1]

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
14,CheXpert-v1.0-small/train/patient00008/study1/...,Male,81,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
15,CheXpert-v1.0-small/train/patient00008/study2/...,Male,81,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,1.0
26,CheXpert-v1.0-small/train/patient00011/study2/...,Female,19,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0
27,CheXpert-v1.0-small/train/patient00011/study10...,Female,19,Frontal,AP,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
28,CheXpert-v1.0-small/train/patient00011/study9/...,Female,19,Frontal,AP,NaN,1.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223397,CheXpert-v1.0-small/train/patient64527/study1/...,Male,85,Frontal,AP,NaN,NaN,-1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0
223401,CheXpert-v1.0-small/train/patient64531/study1/...,Female,57,Frontal,AP,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,-1.0,1.0,1.0,NaN,NaN,1.0
223403,CheXpert-v1.0-small/train/patient64533/study1/...,Male,75,Frontal,AP,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
223407,CheXpert-v1.0-small/train/patient64536/study2/...,Female,61,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
